In [ ]:
import polars as pl
import polars_ds as pld
import numpy as np

In [ ]:
def get_random_data(size:int=2_000) -> pl.DataFrame: 
    df = pl.DataFrame({
        "id": range(size), 
    }).with_columns(
        pl.col("id").cast(pl.UInt64),
        pl.col("id").stats.sample_uniform(low=0.0, high=1.0).alias("feature_1"),
        pl.col("id").stats.sample_uniform(low=0.0, high=1.0).alias("feature_2"),
        pl.col("id").stats.sample_uniform(low=0.0, high=1.0).alias("feature_3"),
        pl.col("id").mod(3).alias("actuals")
    ).with_columns(
        pl.col("val1").stats.rand_null(0.2).alias("val1_nulls")
    )
    return df

    # return df.select(
    #     pl.col("id"),
    #     pl.col("id").num.query_radius_ptwise(
    #         pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
    #         r = 0.1, 
    #         dist = "l2", # actually this is squared l2
    #         parallel = True
    #     ).list.slice(offset=1).alias("friends"),
    # )


In [ ]:
import stumpy
import pandas as pd
from scipy.signal import fftconvolve

df = pld.random_data(100_000) # get_random_data(500_000)
df.head()
ts = pd.Series(df["feature_1"])

In [ ]:
a = fftconvolve(df["feature_1"], [-1, 0, 1], mode="same")
len(a)

In [ ]:

fftconvolve(df["feature_1"], [-1, 0, 1], "same")

In [ ]:
df = pl.DataFrame({
    "a": [5, 6, 7, 8, 9]
})

df.select(
    pl.col("a").num.convolve([1, 0, -1], mode="valid"),
)

In [ ]:
df.select(
    pl.col("feature_1").shift(5).alias("s1"),
)

In [ ]:
df.select(
    pl.col("feature_1").cum_sum().alias("s1"),
    pl.col("feature_1").pow(2).cum_sum().alias("s2"),
    (pl.col("feature_1").pow(2) -  pl.col("feature_1")).cum_sum().alias("s3"),
).with_columns(
    (pl.col("s2") - pl.col("s1")).alias("s4")
)

In [ ]:
# %%timeit
df.select(
    pl.col("feature_1").num.matrix_profile(m = 10, parallel=False)
).unnest("feature_1").with_columns(
    pl.col("squared_znorm").sqrt()
).head(5) 

In [ ]:
%%timeit
df.select(
    pl.col("feature_1").num.matrix_profile2(m = 30, parallel=False)
).unnest("feature_1").with_columns(
    pl.col("squared_znorm").sqrt()
).head(5) # 100k size data, window size = 10, both compiled and parallel

In [ ]:
%%timeit
mp = stumpy.stump(ts, 10)
mp[:5, :]